#### Wildfire Severity Prediction using Raster Tools and SciKit-Learn

## 1. Data Collection

## 2. Building the Dataset

#### The script used to build the dataset is located in the following path: Fire_Prediction/build_mtbs_dataframe.py

##### Once the data has been collected, the dataset can be built.  The files must be arranged in specific folders for the script to work.  The structure is as follows:

> <table><th>Feature</th><th>Folder</th></tr>
<tr><td>MTBS Data</td><td>data/MTBS_Data</td></tr>
<tr><td>DEM Data</td><td>data/terrain</td></tr>
<tr><td>GridMet Climate Data</td><td>data/FeatureData/gridmet</td></tr>
<tr><td>DayMet Climate Data</td><td>data/FeatureData/daymet</td></tr>
<tr><td>AdaptWest Climate Data</td><td>data/FeatureData/adaptwest</td></tr>
<tr><td>Landfire Fuel Data</td><td>data/FeatureData/landfire</td></tr>
<tr><td>Biomass Data</td><td>data/FeatureData/biomass</td></tr>

##### The script will generate a Dask dataframe in Parquet format.  The dataframe will have 20 columns, 19 of which are features and 1 of which is the target variable.  The features are as follows:

> <table><th>Feature</th><th>Column Name</th><th>Temporal Range</th></tr>
<tr><td>MTBS Severity Rating</td><td>mtbs</td><td>Const</td></tr>
<tr><td>MTBS Fire Year</td><td>year</td><td>Const</td></tr>
<tr><td>DEM elevation</td><td>dem</td><td>Const</td></tr>
<tr><td>DEM slope</td><td>dem_slope</td><td>Const</td></tr>
<tr><td>DEM aspect</td><td>dem_aspect</td><td>Const</td></tr>
<tr><td>DEM hillshade</td><td>hillshade</td><td>Const</td></tr>
<tr><td>GridMet Drought Index</td><td>gm_pdsi</td><td>Weekly avg</td></tr>
<tr><td>GridMet Solar Radiation</td><td>gm_srad</td><td>Weekly avg</td></tr>
<tr><td>GridMet Vapor Pressure</td><td>gm_vpd</td><td>Weekly avg</td></tr>
<tr><td>DayMet Temp Max</td><td>dm_tmax</td><td>Monthly max</td></tr>
<tr><td>DayMet Temp Min</td><td>dm_tmin</td><td>Monthly min</td></tr>
<tr><td>AdaptWest Mean Annual Temp</td><td>aw_mat</td><td>Yearly avg</td></tr>
<tr><td>AdaptWest Mean Temp Warmest Month</td><td>aw_mwmt</td><td>Avg of 1 month</td></tr>
<tr><td>AdaptWest Mean Temp Coldest Month</td><td>aw_mcmt</td><td>Avg of 1 month</td></tr>
<tr><td>AdaptWest Temp Difference</td><td>aw_td</td><td>Diff of mwmt and mcmt</td></tr>
<tr><td>Landfire Vegetation Type</td><td>landfire_fvt</td><td>2020 update</td></tr>
<tr><td>Landfire Fuel Model</td><td>landfire_fbfm40</td><td>2020 update</td></tr>
<tr><td>Biomass Annuals</td><td>biomass_afg</td><td>Yearly avg</td></tr>
<tr><td>Biomass Perennials</td><td>biomass_pfg</td><td>Yearly avg</td></tr>
<tr><td>Normalized Difference Vegetation Index</td><td>ndvi</td><td>Weekly avg</td></tr>

## 3. Model Training

## 4. Model Evaluation